In [1]:
import importlib
import mysklearn
importlib.reload(mysklearn)

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyNaiveBayesClassifier
from mysklearn.myclassifiers import MyDecisionTreeClassifier
from mysklearn.myclassifiers import MyRandomForestClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

# Introduction

For this project, we used a fully synthetic dataset from Kaggle. It contains mostly continuous data. It has 15 total attributes and 10,000 instances. We tried to classify if a crop yield was Low, Medium, or High as labels because there was no existing attribute appropriate for prediction.

(findings here)
(best performing classifier)

# Data Analysis

Data information: (details here)



## Relevant Summary Statistics

## Data Visualizations

(code cell below for function calls to display graphs?)

# Classification Results

# Conclusion

# Acknowledgements